In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import torch
import pylab
import torch.nn.functional as F
import torchvision.models as models
import random
import numpy as np
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import unicodedata
import string
import time
import math
import matplotlib.ticker as ticker
import torch.utils.data as data_utils

torch.manual_seed(1)

In [3]:
## Helper functions

### File IO
def find_files(path): return glob.glob(path)

def read_lines(filename):
    lines = open(filename, encoding='utf-8').read() # remove newline
    return lines

def input2Tensor(line):
    tensor = torch.zeros(1, len(line), n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[0][li][char_to_index[letter]] = 1 ## (batch, seq, n_letters)
    return tensor

In [12]:
## Data Loading
TEXT_PATH = 'wiki/enwik8_tail_1000'

lines = ""
for filename in find_files(TEXT_PATH):
    lines = read_lines(filename)

all_letters = list(set(lines))
data_size, n_letters = len(lines), len(all_letters)

char_to_index = {ch : i for i, ch in enumerate(all_letters)}
index_to_char = {i : ch for i, ch in enumerate(all_letters)}

In [18]:
## Models

### input: (seq,  batch, input_size)
### hidden: (num_layers * direction, batch, hidden_size)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.o2o = nn.Linear(output_size, output_size)
        self.softmax = nn.LogSoftmax()
        self.dropmax = nn.Dropout(0.1)
                
    def forward(self, input, hidden):
        
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output_combined = torch.cat((output, hidden), 1)
        output = self.o2o(output)
        output = self.dropmax(output)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

    
    
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, num_layers):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers) 
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.output_size = output_size
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        output, hidden = self.lstm(input, hidden)
        
        output = F.log_softmax(self.h2o(hidden[0][-1]))
        return output, hidden
    
    def initHidden(self):
        return (Variable(torch.randn(self.num_layers, self.batch_size, self.hidden_size)),
                Variable(torch.randn(self.num_layers, self.batch_size, self.hidden_size)))

In [22]:
sequence = 20


epoch = 3
print_every = 10
plot_every = 50


inputs = []
targets = []
for idx in range(0, data_size - sequence):
    #print (lines[idx : idx + sequence], " : ", lines[idx + sequence])
    inputs.append(input2Tensor(lines[idx : idx + sequence]))
    targets.append(char_to_index[lines[idx + sequence]])      

In [23]:
inputs = torch.cat(inputs, 0)
targets = torch.LongTensor(targets)
dataset = torch.utils.data.TensorDataset(inputs, targets)

In [25]:
batch_size = 100
### Define Model

num_layers = 3
hidden_size = 128
input_size = n_letters
output_size = n_letters


all_losses = []
total_loss = 0

rnn = nn.DataParallel(LSTM(n_letters, hidden_size, n_letters, batch_size, num_layers))


### loss 
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=0.001)


### Training
def train(line_tensor, target_tensor):
    ## h_0, c_0
    hidden = (Variable(torch.randn(num_layers, batch_size, hidden_size)),
            Variable(torch.randn(num_layers, batch_size, hidden_size)))
    
    optimizer.zero_grad()
    loss = 0
    output, hidden = rnn(line_tensor, hidden)
    
    #print (target_tensor.shape)
    loss += criterion(output,  target_tensor)
    loss.backward()
    optimizer.step()

    return output, loss.data[0] / line_tensor.size()[0] 

print ("Start Training")

                                      
for epoch_iter in range(0, epoch):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for iter, data in enumerate(dataloader, 0):
        inputTensor, targetTensor = data
        
        if inputTensor.shape[0] != batch_size:
            print ("not enough batch, break")
            break
        
        inputTensor = inputTensor.view(sequence, batch_size, n_letters)
        
        inputTensor, targetTensor = Variable(inputTensor, requires_grad=True), Variable(targetTensor)
        output, loss = train(inputTensor, targetTensor)
        
        
        
        total_loss += loss
        if iter % print_every == 0:
            print ("Epoch : {}, Iteration {}, Loss per {} :  {}".format(epoch_iter, iter, print_every,  loss))
        if iter % plot_every == 0:
            all_losses.append(total_loss / plot_every)
            total_loss = 0


Start Training
Epoch : 0, Iteration 0, Loss per 10 :  0.2703571915626526


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch : 0, Iteration 10, Loss per 10 :  0.22966250777244568
Epoch : 0, Iteration 20, Loss per 10 :  0.17656561732292175
Epoch : 0, Iteration 30, Loss per 10 :  0.1785702258348465
Epoch : 0, Iteration 40, Loss per 10 :  0.1730126142501831
Epoch : 0, Iteration 50, Loss per 10 :  0.17196720838546753
Epoch : 0, Iteration 60, Loss per 10 :  0.1706988662481308
Epoch : 0, Iteration 70, Loss per 10 :  0.17898812890052795
Epoch : 0, Iteration 80, Loss per 10 :  0.17518314719200134
Epoch : 0, Iteration 90, Loss per 10 :  0.17538107931613922
Epoch : 0, Iteration 100, Loss per 10 :  0.16313430666923523
Epoch : 0, Iteration 110, Loss per 10 :  0.17111876606941223
Epoch : 0, Iteration 120, Loss per 10 :  0.18074336647987366
Epoch : 0, Iteration 130, Loss per 10 :  0.17808860540390015
Epoch : 0, Iteration 140, Loss per 10 :  0.16690663993358612
Epoch : 0, Iteration 150, Loss per 10 :  0.17105980217456818
Epoch : 0, Iteration 160, Loss per 10 :  0.16529373824596405
Epoch : 0, Iteration 170, Loss per 1

Epoch : 1, Iteration 140, Loss per 10 :  0.1775980144739151
Epoch : 1, Iteration 150, Loss per 10 :  0.17563840746879578
Epoch : 1, Iteration 160, Loss per 10 :  0.15753671526908875
Epoch : 1, Iteration 170, Loss per 10 :  0.1714009791612625
Epoch : 1, Iteration 180, Loss per 10 :  0.175504669547081
Epoch : 1, Iteration 190, Loss per 10 :  0.17257457971572876
Epoch : 1, Iteration 200, Loss per 10 :  0.18143771588802338
Epoch : 1, Iteration 210, Loss per 10 :  0.16439132392406464
Epoch : 1, Iteration 220, Loss per 10 :  0.17676718533039093
Epoch : 1, Iteration 230, Loss per 10 :  0.16900727152824402
Epoch : 1, Iteration 240, Loss per 10 :  0.17109271883964539
Epoch : 1, Iteration 250, Loss per 10 :  0.17707495391368866
Epoch : 1, Iteration 260, Loss per 10 :  0.16843447089195251
Epoch : 1, Iteration 270, Loss per 10 :  0.17231570184230804
Epoch : 1, Iteration 280, Loss per 10 :  0.1831255853176117
Epoch : 1, Iteration 290, Loss per 10 :  0.1729181706905365
Epoch : 1, Iteration 300, Loss

Epoch : 2, Iteration 270, Loss per 10 :  0.18018914759159088
Epoch : 2, Iteration 280, Loss per 10 :  0.1705566942691803
Epoch : 2, Iteration 290, Loss per 10 :  0.17589163780212402
Epoch : 2, Iteration 300, Loss per 10 :  0.17702490091323853
Epoch : 2, Iteration 310, Loss per 10 :  0.17562493681907654
Epoch : 2, Iteration 320, Loss per 10 :  0.17313405871391296
Epoch : 2, Iteration 330, Loss per 10 :  0.1731942594051361
Epoch : 2, Iteration 340, Loss per 10 :  0.1615360677242279
Epoch : 2, Iteration 350, Loss per 10 :  0.1628122627735138
Epoch : 2, Iteration 360, Loss per 10 :  0.1832326352596283
Epoch : 2, Iteration 370, Loss per 10 :  0.16902202367782593
Epoch : 2, Iteration 380, Loss per 10 :  0.16987909376621246
Epoch : 2, Iteration 390, Loss per 10 :  0.168397918343544
Epoch : 2, Iteration 400, Loss per 10 :  0.17869079113006592
Epoch : 2, Iteration 410, Loss per 10 :  0.16926078498363495
Epoch : 2, Iteration 420, Loss per 10 :  0.1725364774465561
Epoch : 2, Iteration 430, Loss p

In [28]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(np.exp(preds)).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def sample_text(line, sequence, temperature=1.0):
    hidden = (Variable(torch.zeros(num_layers, 1, hidden_size)),
              Variable(torch.zeros(num_layers, 1, hidden_size)))

    string = randomChoice(line, sequence+1)
    input = Variable(lineToTensor(string))
    output_string = string

    print ("Starting with : " + string)
    
    for i in range(max_sample_length):
        output, hidden = rnn(input, hidden)
        #topv, topi = output[0].data.topk(1)
        #print (output[0].data.numpy())
        topi = sample(output[0].data.numpy(), temperature)
        #topi = topi[0]

        if topi == n_letters - 1:
            break
        else:
            letter = all_letters[topi]
            string = string[1:] + letter
            output_string += letter
        input = Variable(lineToTensor(string))

    print ("Generating Text: \n\n\n")
    return output_string
        



max_sample_length = 800
print(sample_text(warm_file, sequence, temperature=0.7))

NameError: name 'randomChoice' is not defined